# Albatross — Expert notebook (WPS + programmatic usage)

This notebook provides a practical, developer-oriented walkthrough for running **Albatross** via WPS and (optionally) locally.

In [ ]:
import urllib.parse
import requests
import re
from pathlib import Path

import pandas as pd
import matplotlib.pyplot as plt


## WPS call

Set your endpoint URL and build an `Execute` request (KVP encoding).

In [ ]:
WPS_URL = "https://<your-wps-host>/wps"  # <-- change me
INPUT_FILE_URL = "https://raw.githubusercontent.com/climateintelligence/albatross/main/albatross/data/APGD_Como_ppt.txt"

params = {
    "service": "WPS",
    "version": "1.0.0",
    "request": "Execute",
    "identifier": "Drought",
    "datainputs": ";".join([
        f"input_file={INPUT_FILE_URL}",
        "target_variable=ZRB",
        "index=ONI",
        "glo_var=SST",
        "phase_mode=2",
        "months=3",
        "training_start_year=1951",
        "training_end_year=2015",
        "target_season=1",
    ]),
}
url = f"{WPS_URL}?{urllib.parse.urlencode(params)}"
print(url)


In [ ]:
resp = requests.get(url, timeout=300)
print("HTTP status:", resp.status_code)
text = resp.text
print(text[:800])

if resp.status_code != 200 or "ProcessSucceeded" not in text:
    m = re.search(r"<(?:\w+:)?ExceptionText>(.*?)</(?:\w+:)?ExceptionText>", text, re.DOTALL)
    if m:
        raise RuntimeError("WPS error: " + re.sub(r"\s+", " ", m.group(1)).strip())
    raise RuntimeError("WPS execution did not succeed (inspect response above).")

print("✅ WPS ProcessSucceeded")


## Extract output references (if any)

In [ ]:
hrefs = re.findall(r'href="([^"]+)"', text)
hrefs = [h for h in hrefs if h.startswith("http")]
print("Found hrefs:", hrefs)


## Local (developer) use

If you have installed the repo (`pip install -e .`), you can inspect the process class and its inputs/outputs.

In [ ]:
from albatross.processes.wps_drought import Drought

proc = Drought()
print("Process identifier:", proc.identifier)
print("Inputs:", [i.identifier for i in proc.inputs])
print("Outputs:", [o.identifier for o in proc.outputs])
